จาก ep ที่แล้ว

# 0. Magic Commands


In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# 1. Import Library

In [2]:
!curl -s https://course.fast.ai/setup/colab | bash

Updating fastai...
Done.


In [0]:
from fastai import *
from fastai.vision import *
from fastai.metrics import accuracy

import torch.optim as optim

# 2. ข้อมูล



In [0]:
path = untar_data(URLs.IMAGENETTE_160)
path_images = path
filenames = get_image_files(path_images)

# 3. เตรียมข้อมูล

In [0]:
batchsize = 64
# np.random.seed(0)
# regex_pattern = r'/([^/]+)_\d+.jpg$'

# databunch = ImageDataBunch.from_name_re(path_images, filenames, 
#                                    regex_pattern, ds_tfms=get_transforms(), 
#                                    size=224, bs=batchsize).normalize(imagenet_stats)

In [0]:
il = ImageList.from_folder(path_images)

In [0]:
sd = il.split_by_rand_pct()

In [0]:
ll = sd.label_from_folder()

In [0]:
tll = ll.transform(get_transforms(), size=160)

In [0]:
databunch = tll.databunch(path_images, num_workers=8, bs=batchsize).normalize()

In [11]:
databunch

ImageDataBunch;

Train: LabelList (10716 items)
x: ImageList
Image (3, 160, 160),Image (3, 160, 160),Image (3, 160, 160),Image (3, 160, 160),Image (3, 160, 160)
y: CategoryList
n02979186,n02979186,n02979186,n02979186,n02979186
Path: /root/.fastai/data/imagenette-160;

Valid: LabelList (2678 items)
x: ImageList
Image (3, 160, 160),Image (3, 160, 160),Image (3, 160, 160),Image (3, 160, 160),Image (3, 160, 160)
y: CategoryList
n03028079,n03888257,n03000684,n03425413,n03394916
Path: /root/.fastai/data/imagenette-160;

Test: None

# 4. สร้างโมเดล


In [0]:
class Optimizer():
    def __init__(self, params, steppers, **defaults):
        # generator? to list
        self.param_groups = list(params)
        # make sure param_groups is list of list
        if not isinstance(self.param_groups[0], dict): self.param_groups = [{'params': self.param_groups}]
        self.hypers = [{**defaults} for p in self.param_groups]
        self.steppers = listify(steppers)

    def grad_params(self):
        return [(p, hyper) for pg, hyper in zip(self.param_groups, self.hypers) for p in pg if p.grad is not None]
    
    def zero_grad(self):
        for p, hyper in self.grad_params():
            p.grad.detach_()
            p.grad.zero_()

    def step(self):
        for p, hyper in self.grad_params(): compose(p, self.steppers, **hyper)


In [0]:
def sgd_step(p, lr, **kwargs):
    p.data.add_(-lr, p.grad.data)
    return p

In [0]:
opt_func = partial(Optimizer, steppers=[sgd_step])

In [0]:
learner = cnn_learner(databunch, models.resnet34, metrics=accuracy, opt_func=opt_func)
learner.unfreeze()

# 5. เริ่มต้นเทรนโมเดล


In [34]:
learner.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy,time


AttributeError: ignored

# 8. สรุป


เปรียบเทียบ

# Credit 

* https://course.fast.ai/videos/?lesson=11